### **Step 1: Extract Data from API**

In [1]:
import requests
import polars as pl

series_ids = ["CPIAUCSL", "FPCPITOTLZGUSA", "UNRATE", "GDP", "GDPC1", "FEDFUNDS", "WM2NS", "DEXUSEU"]

def fetch_series(series_id):
    params = {
        "series_id": series_id,
        "realtime_start": "1900-01-01",
        "api_key": "d27ebbc022a3f8d1a5f9eb6918560e09",
        "file_type": "json"
    }
    response = requests.get("https://api.stlouisfed.org/fred/series/observations", params=params)
    response.raise_for_status()  # Raise an exception for bad status codes
    data = response.json()
    df = pl.DataFrame(data["observations"]).select([
        pl.col("date").cast(pl.Date).alias("date"),
        pl.col("value").str.replace(".", '0').cast(pl.Float64).alias("value")
    ])
    df = df.select([
        "*",
        pl.lit(series_id).alias("series_id")
    ])
    return df

dfs = [fetch_series(series_id) for series_id in series_ids]

# Concatenate the DataFrames
fact_Data = pl.concat(dfs)

fact_Data

date,value,series_id
date,f64,str
1947-01-01,1.48,"""CPIAUCSL"""
1947-02-01,1.62,"""CPIAUCSL"""
1947-03-01,2.0,"""CPIAUCSL"""
1947-04-01,2.0,"""CPIAUCSL"""
1947-05-01,1.95,"""CPIAUCSL"""
…,…,…
2024-05-06,0.0779,"""DEXUSEU"""
2024-05-07,0.0767,"""DEXUSEU"""
2024-05-08,0.0755,"""DEXUSEU"""


### **Step 2: Create Dimension Table**

In [2]:
url_dict = {series_id: f"https://fred.stlouisfed.org/series/{series_id}" for series_id in series_ids}

units_dict = {
    "CPIAUCSL": "Index 1982-1984=100, Seasonally Adjusted",
    "FPCPITOTLZGUSA": "Percent, Not Seasonally Adjusted",
    "UNRATE": "Percent, Seasonally Adjusted",
    "GDP": "Billions of Dollars, Seasonally Adjusted Annual Rate",
    "GDPC1": "Billions of Chained 2017 Dollars, Seasonally Adjusted Annual",
    "FEDFUNDS": "Percent, Not Seasonally Adjusted",
    "WM2NS": "Billions of Dollars, Not Seasonally Adjusted",
    "DEXUSEU": "U.S. Dollars to One Euro, Not Seasonally Adjusted"
}

frequency_dict = {
    "CPIAUCSL": "Monthly",
    "FPCPITOTLZGUSA": "Annual",
    "UNRATE": "Monthly",
    "GDP": "Quarterly",
    "GDPC1": "Quarterly",
    "FEDFUNDS": "Monthly",
    "WM2NS": "Weekly, Ending Monday",
    "DEXUSEU": "Daily"
}

name_dict = {
    "CPIAUCSL": "Consumer Price Index for All Urban Consumers: All Items in U.S. City Average",
    "FPCPITOTLZGUSA": "Inflation, consumer prices for the United States",
    "UNRATE": "Unemployment Rate",
    "GDP": "Gross Domestic Product",
    "GDPC1": "Real Gross Domestic Product",
    "FEDFUNDS": "Federal Funds Effective Rate",
    "WM2NS": "M2",
    "DEXUSEU": "U.S. Dollars to Euro Spot Exchange Rate"
}

dim_SeriesDesc = pl.DataFrame({
    "series_id": series_ids,
    "name": [name_dict[series_id] for series_id in series_ids],
    "unit": [units_dict[series_id] for series_id in series_ids],
    "frequency": [frequency_dict[series_id] for series_id in series_ids],
    "url": [url_dict[series_id] for series_id in series_ids]
})

dim_SeriesDesc

series_id,name,unit,frequency,url
str,str,str,str,str
"""CPIAUCSL""","""Consumer Price…","""Index 1982-198…","""Monthly""","""https://fred.s…"
"""FPCPITOTLZGUSA…","""Inflation, con…","""Percent, Not S…","""Annual""","""https://fred.s…"
"""UNRATE""","""Unemployment R…","""Percent, Seaso…","""Monthly""","""https://fred.s…"
"""GDP""","""Gross Domestic…","""Billions of Do…","""Quarterly""","""https://fred.s…"
"""GDPC1""","""Real Gross Dom…","""Billions of Ch…","""Quarterly""","""https://fred.s…"
"""FEDFUNDS""","""Federal Funds …","""Percent, Not S…","""Monthly""","""https://fred.s…"
"""WM2NS""","""M2""","""Billions of Do…","""Weekly, Ending…","""https://fred.s…"
"""DEXUSEU""","""U.S. Dollars t…","""U.S. Dollars t…","""Daily""","""https://fred.s…"


### **Step 3: Load Data to a DuckDB Database**

In [36]:
import duckdb
%load_ext magic_duckdb

con = duckdb.connect('data.db') #Connect to .db File
%dql -co con

con.sql("CREATE OR REPLACE TABLE fact_FRED AS SELECT * FROM fact_Data;")
con.sql("CREATE OR REPLACE TABLE dim_SRDESC AS SELECT * FROM dim_SeriesDesc;")

The magic_duckdb extension is already loaded. To reload it, use:
  %reload_ext magic_duckdb


In [32]:
con.sql('select table_catalog,table_schema,table_name,table_type from information_schema.tables')

┌───────────────┬──────────────┬────────────┬────────────┐
│ table_catalog │ table_schema │ table_name │ table_type │
│    varchar    │   varchar    │  varchar   │  varchar   │
├───────────────┼──────────────┼────────────┼────────────┤
│ data          │ main         │ dim_SRDESC │ BASE TABLE │
│ data          │ main         │ fact_FRED  │ BASE TABLE │
└───────────────┴──────────────┴────────────┴────────────┘

In [37]:
%%dql
SELECT 
    f.*
    ,d.name
FROM fact_FRED f
INNER JOIN dim_SRDESC d ON f.series_id = d.series_id;

,date,value,series_id,name
0,1947-01-01,1.4800,CPIAUCSL,Consumer Price Index for All Urban Consumers: ...
1,1947-02-01,1.6200,CPIAUCSL,Consumer Price Index for All Urban Consumers: ...
2,1947-03-01,2.0000,CPIAUCSL,Consumer Price Index for All Urban Consumers: ...
3,1947-04-01,2.0000,CPIAUCSL,Consumer Price Index for All Urban Consumers: ...
4,1947-05-01,1.9500,CPIAUCSL,Consumer Price Index for All Urban Consumers: ...
...,...,...,...,...
84154,2024-05-06,0.0779,DEXUSEU,U.S. Dollars to Euro Spot Exchange Rate
84155,2024-05-07,0.0767,DEXUSEU,U.S. Dollars to Euro Spot Exchange Rate
84156,2024-05-08,0.0755,DEXUSEU,U.S. Dollars to Euro Spot Exchange Rate
84157,2024-05-09,0.0771,DEXUSEU,U.S. Dollars to Euro Spot Exchange Rate


### **Step 4: Save Tables to CSV**

In [81]:
df_arrow = con.sql('select table_name from information_schema.tables').arrow() # Extract to Arrow DF
df_table_names = pl.DataFrame(df_arrow).get_column('table_name') # Convert Arrow to Polars DF
print(df_table_names)

for table_name in df_table_names:
    con.sql(f'''COPY {table_name} TO {table_name + '.csv'}''')

shape: (2,)
Series: 'table_name' [str]
[
	"dim_SRDESC"
	"fact_FRED"
]


In [82]:
con.close()